In [27]:
__author__ = "Mark Connolly"

from datetime import datetime, timedelta

from tidepool_data_science_models.models.simple_metabolism_model import SimpleMetabolismModel

from tidepool_data_science_simulator.models.simulation import Simulation, TargetRangeSchedule24hr
from tidepool_data_science_simulator.models.controller import DoNothingController, LoopController
from tidepool_data_science_simulator.models.patient import VirtualPatient
from tidepool_data_science_simulator.models.pump import ContinuousInsulinPump
from tidepool_data_science_simulator.models.sensor import IdealSensor

from tidepool_data_science_simulator.makedata.make_simulation import get_canonical_simulation
from tidepool_data_science_simulator.makedata.make_controller import get_canonical_controller_config
from tidepool_data_science_simulator.makedata.make_patient import (
  DATETIME_DEFAULT, get_canonical_risk_patient_config, get_canonical_risk_pump_config,
    get_canonical_sensor_config
)

from tidepool_data_science_simulator.models.events import BolusTimeline, CarbTimeline
from tidepool_data_science_simulator.models.measures import Bolus, Carb, TargetRange
import json
from loop_to_python_api.api import get_loop_recommendations

t0, patient_config = get_canonical_risk_patient_config(start_glucose_value=250)
t0, sensor_config = get_canonical_sensor_config(start_value=250)
t0, controller_config = get_canonical_controller_config()
t0, pump_config = get_canonical_risk_pump_config()

bolus_timeline = BolusTimeline(datetimes=[t0], events=[Bolus(1.0, "U")])
patient_config.bolus_event_timeline = bolus_timeline
pump_config.bolus_event_timeline = bolus_timeline

true_carb_timeline = CarbTimeline(datetimes=[t0], events=[Carb(20.0, "U", 180)])
patient_config.carb_event_timeline = true_carb_timeline
reported_carb_timeline = CarbTimeline(datetimes=[t0], events=[Carb(25.0, "U", 240)])
pump_config.carb_event_timeline = reported_carb_timeline

pump = ContinuousInsulinPump(pump_config, t0)
sensor = IdealSensor(t0, sensor_config)

controller = LoopController(t0, controller_config)

vp = VirtualPatient(
    time=DATETIME_DEFAULT,
    pump=pump,
    sensor=sensor,
    metabolism_model=SimpleMetabolismModel,
    patient_config=patient_config
)

loop_input = controller.prepare_inputs(virtual_patient=vp)

sim_id = "test_swift_api"
sim = Simulation(
    time=t0,
    duration_hrs=24,
    virtual_patient=vp,
    controller=controller,
    sim_id=sim_id
)

loop_input = sim.controller.prepare_inputs(virtual_patient=vp)
print(loop_input['time_to_calculate_at'])
print(loop_input['glucose_values'])

for _ in range(0, 20):
    sim.step()
    sim.store_state()

loop_input = sim.controller.prepare_inputs(virtual_patient=vp)
print(loop_input['time_to_calculate_at'])
print(loop_input['glucose_values'])

2019-08-15 12:00:00
[250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 250.0]
2019-08-15 13:40:00


AttributeError: dlsym(0x86e418f0, run): symbol not found